We are using this notebook to perform the testing

In [ ]:
# Importing required libraries
import tensorflow as tf
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import cv2 as cv
import os

In [3]:
file_path = r"" # Provide the folder path
pathlib_file = pl.Path(file_path)
class_names = np.array(sorted(item.name for item in pathlib_file.glob("*")))

In [14]:
model = load_model(r"")  # Provide the model path

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 trainable11 (Conv2D)           (None, 300, 300, 32  416         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 trainable21 (Conv2D)           (None, 300, 300, 32  896         ['input_1[0][0]']                
                                )                                                             

                                                                                                  
 add_2 (Add)                    (None, 300, 300, 32  0           ['leaky_re_lu_4[0][0]',          
                                )                                 'leaky_re_lu_5[0][0]']          
                                                                                                  
 add_3 (Add)                    (None, 300, 300, 32  0           ['leaky_re_lu_6[0][0]',          
                                )                                 'leaky_re_lu_7[0][0]']          
                                                                                                  
 add_4 (Add)                    (None, 300, 300, 32  0           ['leaky_re_lu_8[0][0]',          
                                )                                 'leaky_re_lu_9[0][0]']          
                                                                                                  
 add_5 (Ad

 trainable114 (Conv2D)          (None, 37, 37, 256)  295168      ['max_pooling2d_2[0][0]']        
                                                                                                  
 nontrainable114 (Conv2D)       (None, 37, 37, 256)  295168      ['max_pooling2d_2[0][0]']        
                                                                                                  
 nontrainable124 (Conv2D)       (None, 37, 37, 256)  295168      ['max_pooling2d_2[0][0]']        
                                                                                                  
 nontrainable134 (Conv2D)       (None, 37, 37, 256)  295168      ['max_pooling2d_2[0][0]']        
                                                                                                  
 maximum_2 (Maximum)            (None, 37, 37, 256)  0           ['trainable114[0][0]',           
                                                                  'nontrainable114[0][0]',        
          

In [6]:
def process_img(image, file_name):
    try:
        img = tf.io.decode_image(contents = image, channels=3, expand_animations=False)
        img = tf.image.resize(img, [300, 300])     
        return img
    except tf.errors.InvalidArgumentError as e:
        os.remove(file_name)
def process_data(file_name):
    img = tf.io.read_file(file_name)
    img = process_img(img, file_name)
    img = img/255.0
    return img
AUTOTUNE = tf.data.AUTOTUNE
def configure_inputs_train(data_set):
    data_set = data_set.batch(batch_size=8)
    data_set = data_set.prefetch(buffer_size=AUTOTUNE)
    return data_set

In [7]:
cat_file= pl.Path(file_path + "\\" + "Cat")
dog_file = pl.Path(file_path + "\\" + "Dog")

In [8]:
cat_file_list = list(cat_file.glob("*"))
dog_file_list = list(dog_file.glob("*"))

In [9]:
cat_data_set = tf.data.Dataset.list_files(str(cat_file/"*"), shuffle=False)
dog_data_set = tf.data.Dataset.list_files(str(dog_file/"*"), shuffle=False)

AUTOTUNE = tf.data.AUTOTUNE

cat_data_set = cat_data_set.map(process_data, num_parallel_calls=AUTOTUNE)
dog_data_set = dog_data_set.map(process_data, num_parallel_calls=AUTOTUNE)

cat_data_set = configure_inputs_train(cat_data_set)
dog_data_set = configure_inputs_train(dog_data_set)


In [15]:
cat_predictions = model.predict(cat_data_set)
cat_predictions = cat_predictions[:, 0]
dog_predictions = model.predict(dog_data_set)
dog_predictions = dog_predictions[:, 1]

375/375 [==============================] - 22s 58ms/step


In [17]:
print(f"Correctly predicted {len(np.where(cat_predictions > 0.5)[0])} out of {len(cat_predictions)}")
print(f"Correctly predicted {len(np.where(dog_predictions > 0.5)[0])} out of {len(dog_predictions)}")

Correctly predicted 2945 out of 3000
Correctly predicted 2888 out of 3000


In [18]:
# Displaying the Accuracy
print(f"Cat accuracy: {len(np.where(cat_predictions > 0.5)[0]) /len(cat_predictions)*100}")
print(f"Dog accuracy: {len(np.where(dog_predictions > 0.5)[0]) /len(dog_predictions)*100}")

Cat accuracy: 98.16666666666667
Dog accuracy: 96.26666666666667


In [24]:
net_activation = (np.mean(cat_predictions) + np.mean(dog_predictions))/2

In [25]:
# Displaying the Activation rate
print(f"Cat Activatin Rate: {np.mean(cat_predictions)}")
print(f"Dog Activatin Rate: {np.mean(dog_predictions)}")
print(f"Net Activatin Rate: {net_activation}")

Cat Activatin Rate: 0.794951856136322
Dog Activatin Rate: 0.8449637293815613
Net Activatin Rate: 0.8199577927589417


In [26]:
cat_not_predicted = []
dog_not_predicted = []
for i in range(len(cat_file_list)):
    if cat_predictions[i] < 0.5:
        cat_not_predicted.append((i, cat_predictions[i]))
for i in range(len(dog_file_list)):
    if dog_predictions[i] < 0.5:
        dog_not_predicted.append((i, dog_predictions[i]))

In [27]:
# Displaying the number of wrong predictions in each class
print(len(cat_not_predicted))
print(len(dog_not_predicted))

1842
1164


In [35]:
# Displaying the path of incorrectly classified images
for i in range(0, 40):
    print(cat_file_list[cat_not_predicted[i][0]])

D:\Samsung Prism\Datasets\Replica\Cat\10009.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10019.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10023.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10036.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10047.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10051.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10053.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10060.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10077.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10094.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10096.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10114.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10137.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10138.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10146.jpg
D:\Samsung Prism\Datasets\Replica\Cat\1015.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10152.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10156.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10159.jpg
D:\Samsung Prism\Datasets\Replica\Cat\10162.jpg
D:\Samsung Prism\Datasets\Replica\Cat\101